# Applications Weighted Grading Automation
<a target="_blank" href="https://colab.research.google.com/github/trendinafrica/student_selection_process_automation/blob/main/weighted_grading.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

(*By: [@mahmoud-elmakki](https://github.com/mahmoud-elmakki)*)

This stage is for calculating the average grade of a response from the grades given by multiple evaluatores. Each reviewer gives a general grade for each response.

Also see [main.ipynb](https://github.com/trendinafrica/student_selection_process_automation/blob/main/main.ipynb)

### **How to use (as a developer):**
Just clone the Github repository and get into the business!\
If you have anaconda and yupyter installed locally you can just clone the repory directly on your machine. Elsewise, you can clone it into Google Colab.
(In either case, don't forget to regularly pull and push changes).

### **How to use (as a reviewer):**
If you are on Github now, open this notebook in Google Colab, or clone the whole repo locally, so you can run the cells. In case of running it in Colab, don't forget to save and download the resulting Excel sheet of the processed responses into a local folder.

In [1]:
import numpy as np
import pandas as pd

import os

In [2]:
# Note that you have to download the responses data Excel sheet from Google Drive and put it in the same folder as the code.
# You don't have to do this if you cloned the Github repo (all will be organized in the repo).
 
# TODO: Load data directly from Google Drive.

# Loading students responses data
REVD_STDN_DATA_DIR = './reviewed_responses/Mahmoud_and_Burak/reviewed_dummy_responses_Mahmoud_and_Burak.xlsx'
reviewed_responses = pd.read_excel(REVD_STDN_DATA_DIR)

reviewed_responses['Total Review'] = None

# Just specify folder names - thje code will create the directory
OUTPUT_FOLDER_NAME = "first_grading_round_results"
OUTPUT_DIR = os.path.join(os.getcwd(), OUTPUT_FOLDER_NAME)

if not os.path.exists(OUTPUT_DIR):
    os.mkdir(OUTPUT_DIR)

In [3]:
# Use this dictionary as a reference for column names.

qs_dict = {i: column for i, column in enumerate(reviewed_responses.columns)}
qs_dict

{0: 'Timestamp',
 1: 'Email address',
 2: 'First Name',
 3: 'Last Name',
 4: 'Age',
 5: 'Nationality',
 6: 'In case you are selected, where would you be traveling from to Rwanda?',
 7: 'Gender',
 8: 'What is your highest degree of education?',
 9: 'What is your current career stage?',
 10: 'Which group of studies best describes your background?',
 11: 'Name of current University/Research Institution/Organization (e.g. Department of Biomedical Engineering, CMU-Africa, Kigali, Rwanda)',
 12: 'Field of studies in Undergraduate (completed or ongoing, eg. Neuroscience, Mathematics, Law)',
 13: 'Field of studies in Master’s (if applicable, completed or ongoing, eg. Neuroscience, Mathematics, Law)',
 14: 'Field of studies in PhD/MD degree (if applicable, completed or ongoing, eg. Neuroscience, Mathematics, Law)',
 15: 'Current research focus or research focus of the last research project you were engaged in (if applicable)',
 16: 'Rate your proficiency of computer programming (with any progra

In [4]:
# Used indices of the student responses DataFrame

std_idcs = {
    'email_idx' : 1,
    'firstname_idx' : 2,
    'lastname_idx' : 3,
    'nat_idx' : 5,
    'resid_idx' : 6,
    'cv_idx' : 23,
    'ref' : {
        'first_ref_email_idx' : 25,
        'second_ref_email_idx' : 27
          },
    'flag_idx' : 28,
    'notes_idx' : 29,
    'first_recomm_letter_idx' : 30,
    'second_recomm_letter_idx' : 31,
    'reviewer_1_idx' : 32,
    'reviewer_2_idx' : 33,
    'total_review_idx' : 34
    
}

stdn_str_qs = [std_idcs['email_idx'], std_idcs['firstname_idx'], std_idcs['lastname_idx']]

# Carefully specify names of the columns to be processed (CV and recommendation letters).
reviews_idcs = [std_idcs['reviewer_1_idx'], std_idcs['reviewer_2_idx']]
reviews_ws = [1, 1]

WEIGHTED_REVIEWING = False

if not WEIGHTED_REVIEWING:
    reviews_ws = [1, 1]

# Number of students to be selected.
TOP_N = 5

if TOP_N > len(reviewed_responses):
    TOP_N = len(reviewed_responses)

In [5]:
def column_names_to_indices(df, indices_dict):
    """
    Replaces column names with indices.
    """
    processed_df = df.rename(columns={column: i for i, column in enumerate(indices_dict.values())})

    return processed_df


def indices_to_column_names(df, indices_dict):
    """
    Replaces indices with column names.
    """
    processed_df = df.rename(columns={i: column for i, column in enumerate(indices_dict.values())})

    return processed_df


def calc_review(reviewed_responses, reviews=reviews_idcs, weights=reviews_ws):
    
    for row_index, row in reviewed_responses.iterrows():
        reviews = []
        
        for rev in reviews_idcs:
            review = float(reviewed_responses.iloc[row_index, rev])
            reviews.append(review)
        
        weighted_reviews = [reviews[i] * weights[i] for i in range(len(reviews))]
        total_review = float(round(sum(weighted_reviews) / sum(weights), 2))
        reviewed_responses.iloc[row_index, std_idcs['total_review_idx']] = total_review
            
    return reviewed_responses
 

In [6]:
def main(reviewed_responses):
    
    reviewed_responses = column_names_to_indices(reviewed_responses, qs_dict)
    reviewed_responses = calc_review(reviewed_responses)
    reviewed_responses_sorted = reviewed_responses.sort_values(by = std_idcs['total_review_idx'], ascending = False)
    
    indices_to_column_names(reviewed_responses, qs_dict).to_excel(OUTPUT_FOLDER_NAME + "/reviewed_responses.xlsx")
    
    top_n_responses = reviewed_responses_sorted.iloc[:TOP_N, :]
    
    indices_to_column_names(top_n_responses, qs_dict).to_excel(OUTPUT_FOLDER_NAME + "/top_n_reviewed_responses.xlsx")
    
    return reviewed_responses, top_n_responses

In [7]:
reviewed_responses_final, top_n_responses = main(reviewed_responses)

In [8]:
reviewed_responses_final

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,2024-01-16 08:06:27.194,jakindaodhiambo@gmail.com,Jakinda,Oluoch,25,Kenya,Kenya,Male,Bachelor's degree,Working in industry,...,\tbalaka@strathmore.edu,Wallace Muchiri,wmuchiri@strathmore.edu,NaN,NaN,NaN,NaN,6,9,7.5
1,2024-01-16 14:26:12.656,ejirukomaotomewo@gmail.com,Lily,Otomewo,24,Nigeria,Nigeria,Female,Bachelor's degree,Master student,...,adeoluwaoa@abuad.edu.ng,Dr. Gladys Onyinye ADEOLUWA,adeoluwago@abuad.edu.ng,NaN,NaN,NaN,NaN,8,3,5.5
2,2024-01-16 15:22:53.482,mohamedahmed@aims.edu.gh,Mohamedahmed,Mohamed,29,Sudan,Rwanda,Male,Master's,Master student,...,josephasare@ug.edu.gh,Prof. Pietro Caputo / Università Roma Tre,pietro.caputo@uniroma3.it,NaN,NaN,NaN,NaN,6,5,5.5
3,2024-01-16 15:23:31.832,2567491@students.wits.ac.za,Talha,Niazi,20,South Africa,South Africa,Male,High school degree,Undergraduate,...,vrajamany@redhill.co.za,Nick Smit/Smergos,nick@smergos.com,NaN,NaN,NaN,NaN,8,7,7.5
4,2024-01-16 15:31:57.857,Paul.Fadojutimi@wits.ac.za,Paul,Fadojutimi,42,Nigeria,South Africa,Male,PhD/MD degree,Postdoc,...,Nosipho.Moloto@wits.ac.za,Dr Siziwe Gqoba/Wits University,siziwe.gqoba@wits.ac.za,NaN,NaN,NaN,NaN,5,4,4.5
5,2024-01-16 15:35:37.142,nclement@aims.ac.rw,Clement,NSHIMIYIMANA,30,Rwanda,Rwanda,Male,Master's,Working in industry,...,blaise.tchapnda@aims.ac.rw,mustafa mcisse@aimsammi.org,tbahunde@aimsammi.org,NaN,NaN,NaN,NaN,4,1,2.5
6,2024-01-16 17:51:12.036,ocheab1@gmail.com,Oche,George,34,Nigeria,Nigeria,Male,Master's,PhD,...,cemtrod.ilorin@gmail.com,Dr. John Olanrewaju Afees/Backcock University,olanrewajuj@babcock.edu.ng,NaN,NaN,NaN,NaN,1,6,3.5
7,2024-01-16 18:16:25.739,titilayomiotenaike@gmail.com,TITILAYOMI,OTENAIKE,29,Nigeria,Brazil,Female,Master's,PhD,...,elgionl@gmail.com,Dr Abolaji Samson Olagunju,abolajisamson@icb.usp.br,NaN,NaN,NaN,NaN,2,9,5.5
8,2024-01-16 18:25:48.402,tierryhma@gmail.org,HOUNKONNOU,Mitonsou Tierry,29,Benin,Benin,Male,Master's,PhD,...,anoumou.attiogbe@aims-senegal.org,Eliane Raïssa Fankem,elian.r.fankem@aims-senegal.org,NaN,NaN,NaN,NaN,2,10,6.0
9,2024-01-16 18:38:10.348,ousseynou.mbaye@uadb.edu.sn,Ousseynou,MBAYE,38,Senegal,Senegal,Male,PhD/MD degree,PhD,...,mouhamadoulamine.ba@esp.sn,Babacar Mbaye,babacar.mbaye@uadb.edu.sn,NaN,NaN,NaN,NaN,9,7,8.0


In [9]:
top_n_responses

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
9,2024-01-16 18:38:10.348,ousseynou.mbaye@uadb.edu.sn,Ousseynou,MBAYE,38,Senegal,Senegal,Male,PhD/MD degree,PhD,...,mouhamadoulamine.ba@esp.sn,Babacar Mbaye,babacar.mbaye@uadb.edu.sn,NaN,NaN,NaN,NaN,9,7,8.0
13,2024-01-16 20:07:50.242,daphne.machangara@aims-cameroon.org,Daphne,Machangara,33,Zimbabwe,Zimbabwe,Female,Master's,In search of the next step,...,ihechukwu@aims.ac.za,Arnaud Fadja Nguembang/University of Ferrara,arnaud.nguembafadja@unife.it,NaN,NaN,NaN,NaN,7,9,8.0
0,2024-01-16 08:06:27.194,jakindaodhiambo@gmail.com,Jakinda,Oluoch,25,Kenya,Kenya,Male,Bachelor's degree,Working in industry,...,\tbalaka@strathmore.edu,Wallace Muchiri,wmuchiri@strathmore.edu,NaN,NaN,NaN,NaN,6,9,7.5
17,2024-01-17 12:10:54.634,yahigbm@gmail.com,Youcef,YAHI,29,Algeria,Algeria,Male,Master's,PhD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,6,7.5
3,2024-01-16 15:23:31.832,2567491@students.wits.ac.za,Talha,Niazi,20,South Africa,South Africa,Male,High school degree,Undergraduate,...,vrajamany@redhill.co.za,Nick Smit/Smergos,nick@smergos.com,NaN,NaN,NaN,NaN,8,7,7.5
